In [1]:
# interval arithmetics are used to take into account the numerical errors
R = RIF

var('x a')

o = 6              # the order of polynomial approximations;
assert(o % 2 == 0) # should be even to ensure that the bounds on the remainders
                   # in the Taylor approximations below are correct

maxx = 1 / 30 # alpha = 11 * pi / 24 + x, where -maxx <= x <= maxx

# cleaner print
def myprint(e):
    print(*e, sep='\n')

# return the polynomial with the coefficients evaluated
def eval(p):
    return sum([R(a) * x ^ b for a, b in p.coefficients()])

# get the polynomial without the constant coefficient
def center(p):
    return sum([a * x ^ b for a, b in p.coefficients() if b > 0])

# get the constant coefficient of the polynomial
def const(p):
    return R(p.coefficient(x, n=0))

# given polynomials p[0], p[1], such that p[0](x) <= p[1](x) on [-maxx, maxx],
# and even number newd, return polynomials np[0], np[1] of degree <= newd,
# such that np[0](x) <= p[0](x) <= p[1](x) <= np[1](x) on [-maxx, maxx]
def reduceDegree(p, newd):
    assert(newd % 2 == 0)
    np = [0, 0]
    for k, d in p[0].coefficients():
        if d <= newd:
            np[0] += k * x ^ d
        else:
            np[0] += -abs(k) * maxx ^ (d - newd) * x ^ newd
    for k, d in p[1].coefficients():
        if d <= newd:
            np[1] += k * x ^ d
        else:
            np[1] += abs(k) * maxx ^ (d - newd) * x ^ newd
    return np

# given polynomial p of degree <= 2, return an array of points,
# which contains argmax and argmin of p on [-maxx, maxx]
def getInterestingPoints(p):
    assert(p.degree(x) <= 2)
    if p.degree(x) < 2:
        return [-maxx, maxx]
    e = -p.coefficient(x, n=1) / (2 * p.coefficient(x, n=2))
    if e <= -maxx or maxx <= e: # e is definitely outside [-maxx, maxx]
        return [-maxx, maxx]
    else:
        return [-maxx, e, maxx]

# estimation of the analytic function f(x) on the interval [-maxx, maxx]:
# contains polynomials p[0], p[1] of order <= o, and real numbers b[0], b[1],
# such that b[0] <= p[0](x) <= f(x) <= p[1](x) <= b[1] on [-maxx, maxx]
class Estimator:
    def __init__(self, p, b=None):
        self.p = p
        if b == None:
            self.getBounds()
        else:
            self.b = b
      
    def __add__(self, t):
        return Estimator([eval(self.p[0] + t.p[0]), eval(self.p[1] + t.p[1])])
    
    def __sub__(self, t):
        return Estimator([eval(self.p[0] - t.p[1]), eval(self.p[1] - t.p[0])])
    
    def __neg__(self):
        return Estimator([-self.p[1], -self.p[0]])
    
    # we multiply only estimations for functions with nonnegative values in [-maxx, maxx]
    def __mul__(self, t):
        assert(self.b[0] >= 0 and t.b[0] >= 0)
        res = Estimator([eval(self.p[0] * t.p[0]), eval(self.p[1] * t.p[1])])
        res.reduceDegree()
        return res
    
    def split(self):
        return [[center(self.p[0]), const(self.p[0])],
                [center(self.p[1]), const(self.p[1])]]
    
    def reduceDegree(self):
        self.p = reduceDegree(self.p, o)
        self.getBounds()
        
    def transposed(self):
        return Estimator(self.p[::-1], self.b[::-1])
    
    # print the estimations; bounds b[0] and b[1] are printed in degrees, if inDegs=True
    def print(self, inDegs=False):
        myprint(self.p)
        if inDegs:
            print(R(self.b[0] / pi * 180), R(self.b[1] / pi * 180))
        else:
            print(self.b[0], self.b[1])

    # to get bounds b[0], b[1] for polynomials p[0], p[1],
    # we obtain polynomials p2[0], p2[1] of degree <= 2, such that
    # p2[0](x) <= p[0](x) <= p[1](x) <= p2[1](x) on [-maxx, maxx],
    # and explicitly find the minimum of p2[0] and the maximum of p2[1]
    def getBounds(self):
        p2 = reduceDegree(self.p, 2)
        self.b = [None, None]
        for c in getInterestingPoints(p2[0]):
            val = R(p2[0].subs(x=c))
            self.b[0] = val if self.b[0] == None else self.b[0].min(val)
        for c in getInterestingPoints(p2[1]):
            val = R(p2[1].subs(x=c))
            self.b[1] = val if self.b[1] == None else self.b[1].max(val)

# get the estimator for a constant function f(x) = C
def E(C):
    return Estimator([C, C], [C, C])

# substitute estimator e into Taylor approximations q[0], q[1]
def subs(q, e):
    func = lambda q, args : eval(q.subs(x=args[0], a=args[1]))
    res = Estimator(list(map(func, q, e.split())))
    res.reduceDegree()
    return res

In [2]:
sint = sin(x + a).series(x, o).truncate()
sinf = [sint,
        sint + x ^ o / factorial(o)]
myprint(sinf)

# sinf[0](x, a) <= sin(x + a) <= sinf[1](x, a) for 0 < a < pi / 2, 0 < x + a < pi / 2

# given estimator e of f(x), return the estimator of sin(f(x))
def mysin(e):
    if pi / 2 < e.b[0] and e.b[1] < pi: # sin(x) = sin(pi - x)
        return mysin(E(pi) - e)
    assert(0 < e.b[0] and e.b[1] < pi / 2)
    res = subs(sinf, e)
    return res

1/120*x^5*cos(a) + 1/24*x^4*sin(a) - 1/6*x^3*cos(a) - 1/2*x^2*sin(a) + x*cos(a) + sin(a)
1/720*x^6 + 1/120*x^5*cos(a) + 1/24*x^4*sin(a) - 1/6*x^3*cos(a) - 1/2*x^2*sin(a) + x*cos(a) + sin(a)


In [3]:
cost = cos(x + a).series(x, o).truncate()
cosf = [cost,
        cost + x ^ o / factorial(o)]
myprint(cosf)

# cosf[0](x, a) <= cos(x + a) <= cosf[1](x, a) for 0 < a < pi / 2, 0 < x + a < pi / 2

# given estimator e of f(x), return the estimator of cos(f(x))
def mycos(e):
    if pi / 2 < e.b[0] and e.b[1] < pi: # cos(x) = -cos(pi - x)
        return -mycos(E(pi) - e)
    assert(0 < e.b[0] and e.b[1] < pi / 2)
    res = subs(cosf, e.transposed()) # cos is decreasing on (0, pi / 2)
    return res

-1/120*x^5*sin(a) + 1/24*x^4*cos(a) + 1/6*x^3*sin(a) - 1/2*x^2*cos(a) - x*sin(a) + cos(a)
1/720*x^6 - 1/120*x^5*sin(a) + 1/24*x^4*cos(a) + 1/6*x^3*sin(a) - 1/2*x^2*cos(a) - x*sin(a) + cos(a)


In [4]:
acost = acos(x + a).series(x, o).truncate()
acosd = acos(x).derivative(x, o)
acosb = 85 / 100
acosf = [acost - x ^ o * abs(acosd.subs(x=acosb)) / factorial(o),
         acost]
myprint(acosf)

# acosf[0](x, a) <= acos(x + a) <= acosf[1](x, a) for 0 < a < acosb, 0 < x + a < acosb

# given estimator e of f(x), return the estimator of acos(f(x))
def myacos(e):
    assert(0 < e.b[0] and e.b[1] < acosb)
    res = subs(acosf, e.transposed()) # acos is decreasing
    return res

-174355808000000/1870414552161*sqrt(111)*x^6 - 1/40*x^5*(35*a^4/(-a^2 + 1)^(9/2) + 30*a^2/(-a^2 + 1)^(7/2) + 3/(-a^2 + 1)^(5/2)) - 1/8*x^4*(5*a^3/(-a^2 + 1)^(7/2) + 3*a/(-a^2 + 1)^(5/2)) - 1/6*x^3*(3*a^2/(-a^2 + 1)^(5/2) + 1/(-a^2 + 1)^(3/2)) - 1/2*a*x^2/(-a^2 + 1)^(3/2) - x/sqrt(-a^2 + 1) + arccos(a)
-1/40*x^5*(35*a^4/(-a^2 + 1)^(9/2) + 30*a^2/(-a^2 + 1)^(7/2) + 3/(-a^2 + 1)^(5/2)) - 1/8*x^4*(5*a^3/(-a^2 + 1)^(7/2) + 3*a/(-a^2 + 1)^(5/2)) - 1/6*x^3*(3*a^2/(-a^2 + 1)^(5/2) + 1/(-a^2 + 1)^(3/2)) - 1/2*a*x^2/(-a^2 + 1)^(3/2) - x/sqrt(-a^2 + 1) + arccos(a)


In [5]:
invt = (1 / (x + a)).series(x, o).truncate()
invd = (1 / x).derivative(x, o)
invb = 4 / 10
invf = [invt,
        invt + x ^ o * abs(invd.subs(x=invb)) / factorial(o)]
myprint(invf)

# invf[0](x, a) <= 1 / (x + a) <= invf[1](x, a) for invb < a, invb < x + a < 2 * a

# given estimator e of f(x), return the estimator of 1 / f(x)
def myinv(e):
    assert(invb < e.b[0] and e.b[1] < 2 * const(e.p[0]).min(const(e.p[1])))
    res = subs(invf, e.transposed()) # 1 / x is decreasing when x > 0
    return res

1/a - x/a^2 + x^2/a^3 - x^3/a^4 + x^4/a^5 - x^5/a^6
78125/128*x^6 + 1/a - x/a^2 + x^2/a^3 - x^3/a^4 + x^4/a^5 - x^5/a^6


In [6]:
inv2t = (1 / (x + a)).series(x, o).truncate()
inv2d = (1 / x).derivative(x, o)
inv2b = 12 / 10
inv2f = [inv2t,
        inv2t + x ^ o * abs(inv2d.subs(x=inv2b)) / factorial(o)]
myprint(inv2f)

# inv2f[0](x, a) <= 1 / (x + a) <= inv2f[1](x, a) for inv2b < a, inv2b < x + a < 2 * a

# given estimator e of f(x), return the estimator of 1 / f(x)
def myinv2(e):
    assert(inv2b < e.b[0] and e.b[1] < 2 * const(e.p[0]).min(const(e.p[1])))
    res = subs(inv2f, e.transposed()) # 1 / x is decreasing when x > 0
    return res

1/a - x/a^2 + x^2/a^3 - x^3/a^4 + x^4/a^5 - x^5/a^6
78125/279936*x^6 + 1/a - x/a^2 + x^2/a^3 - x^3/a^4 + x^4/a^5 - x^5/a^6


In [7]:
sqrtt = sqrt(x + a).series(x, o).truncate()
sqrtd = sqrt(x).derivative(x, o)
sqrtb = 15 / 10
sqrtf = [sqrtt - x ^ o * abs(sqrtd.subs(x=sqrtb)) / factorial(o),
        sqrtt]
myprint(sqrtf)

# sqrtf[0](x, a) <= sqrt(x + a) <= sqrtf[1](x, a) for sqrtb < a, sqrtb < x + a < 2 * a

def mysqrt(e):
    assert(sqrtb < e.b[0] and e.b[1] < 2 * const(e.p[0]).min(const(e.p[1])))
    res = subs(sqrtf, e)
    return res

-7/3888*sqrt(3/2)*x^6 + sqrt(a) + 1/2*x/sqrt(a) - 1/8*x^2/a^(3/2) + 1/16*x^3/a^(5/2) - 5/128*x^4/a^(7/2) + 7/256*x^5/a^(9/2)
sqrt(a) + 1/2*x/sqrt(a) - 1/8*x^2/a^(3/2) + 1/16*x^3/a^(5/2) - 5/128*x^4/a^(7/2) + 7/256*x^5/a^(9/2)


In [8]:
dalpha = Estimator([x, x])
dalpha.print()

x
x
-0.03333333333333334? 0.03333333333333334?


In [9]:
alpha = E(11 * pi / 24) + dalpha
alpha.print(inDegs=True)

1*x + 1.439896632895322?
1*x + 1.439896632895322?
80.5901406828973? 84.4098593171028?


In [10]:
q = alpha - E(pi / 12) + myacos(E(1) - mysin(E(2) * alpha)) * E(1 / 2)
q.print(inDegs=True)

-27155.407732377?*x^6 - 254.52788054163?*x^5 - 47.46570482625?*x^4 - 9.514635993752?*x^3 - 2.6714163401324?*x^2 - 0.43887712637302?*x + 1.54608148543621?
-172.09609761616?*x^6 - 254.52788054163?*x^5 - 47.46570482625?*x^4 - 9.514635993752?*x^3 - 2.6714163401324?*x^2 - 0.43887712637302?*x + 1.54608148543621?
87.549400068237? 89.276231682630?


In [11]:
gamma = alpha - E(pi / 12) - myacos(E(1) - mysin(E(2) * alpha)) * E(1 / 2)
gamma.print(inDegs=True)

172.09609761616?*x^6 + 254.52788054163?*x^5 + 47.46570482625?*x^4 + 9.514635993752?*x^3 + 2.6714163401324?*x^2 + 2.43887712637302?*x + 0.810113004756141?
27155.407732377?*x^6 + 254.52788054163?*x^5 + 47.46570482625?*x^4 + 9.514635993752?*x^3 + 2.6714163401324?*x^2 + 2.43887712637302?*x + 0.810113004756141?
41.9040496831646? 51.270318565969?


In [12]:
t = myacos(E(1) + mycos(E(2) * alpha) - mycos(E(2 * pi / 3) - E(2) * alpha + E(2) * q))
t.print(inDegs=True)

-506030.0297091?*x^6 + 1013.996707978?*x^5 + 0.7770005651?*x^4 + 39.18325634813?*x^3 - 0.174598201652?*x^2 + 3.7393664881860?*x + 0.78783798555390?
75052.11762651?*x^6 + 1013.996707978?*x^5 + 0.7770005651?*x^4 + 39.18325634813?*x^3 - 0.174598201652?*x^2 + 3.7393664881860?*x + 0.78783798555390?
37.861645603699? 52.361834163985?


In [13]:
delta = E(pi / 3) - alpha + q - t * E(1 / 2)
delta.print(inDegs=True)

-64681.46654563?*x^6 - 761.526234531?*x^5 - 47.8542051088?*x^4 - 29.10626416782?*x^3 - 2.584117239307?*x^2 - 3.3085603704661?*x + 0.75946341096053?
252842.9187570?*x^6 - 761.526234531?*x^5 - 47.8542051088?*x^4 - 29.10626416782?*x^3 - 2.584117239307?*x^2 - 3.3085603704661?*x + 0.75946341096053?
36.958623669142? 49.755241146086?


In [14]:
X = mysin(t + delta) * myinv(mysin(t))
X.print()

-127978.3951578?*x^6 - 8610.57171075?*x^5 + 1253.002476752?*x^4 - 188.4302943537?*x^3 + 29.35993515758?*x^2 - 5.234000842793?*x + 1.41038635842902?
1.493309532426?e6*x^6 - 8610.57171075?*x^5 + 1253.002476752?*x^4 - 188.4302943537?*x^3 + 29.35993515758?*x^2 - 5.234000842793?*x + 1.41038635842902?
1.2594860995466? 1.6284037998802?


In [15]:
YnumR = E(1) - mysin(delta) * mysin(gamma) * mycos(t) * myinv(mysin(t)) + mysin(delta) * mycos(gamma)
YnumR.print()

-1.226517318686?e6*x^6 + 4039.27658238?*x^5 - 610.320050998?*x^4 + 85.2981754027?*x^3 - 12.40404986602?*x^2 + 1.418839628481?*x + 0.9783646196016?
288675.3260633?*x^6 + 4039.27658238?*x^5 - 610.320050998?*x^4 + 85.2981754027?*x^3 - 12.40404986602?*x^2 + 1.418839628481?*x + 0.9783646196016?
0.9115263236325? 1.0163518829568?


In [16]:
Ynum = mycos(t + E(2) * delta - E(pi / 2)) * YnumR - (mysin(delta) + mycos(gamma)) * mycos(q)
Ynum.print()

-1.478839636912?e6*x^6 + 1982.41887176?*x^5 - 278.119705903?*x^4 + 41.8227133488?*x^3 - 7.55714640801?*x^2 + 2.440058751547?*x + 0.6910928992730?
742117.107953?*x^6 + 1982.41887176?*x^5 - 278.119705903?*x^4 + 41.8227133488?*x^3 - 7.55714640801?*x^2 + 2.440058751547?*x + 0.6910928992730?
0.5973582635366? 0.7670232830068?


In [17]:
Yden = mycos(t + E(2) * delta - E(pi / 2)) * mycos(gamma) - mycos(q)
Yden.print()

-372107.2541170?*x^6 - 525.210414727?*x^5 - 99.6633439532?*x^4 - 26.10972118283?*x^3 - 8.678171551256?*x^2 - 0.416290559252?*x + 0.48627011420287?
357185.2182065?*x^6 - 525.210414727?*x^5 - 99.6633439532?*x^4 - 26.10972118283?*x^3 - 8.678171551256?*x^2 - 0.416290559252?*x + 0.48627011420287?
0.4611292327291? 0.4922568933096?


In [18]:
Y = Ynum * myinv(Yden)
Y.print()

-4.427501585099?e6*x^6 + 13305.6283477?*x^5 + 569.96861820?*x^4 + 286.560732030?*x^3 + 15.1598389616?*x^2 + 6.234592219888?*x + 1.4212119541952?
3.010010143866?e6*x^6 + 13305.6283477?*x^5 + 569.96861820?*x^4 + 286.560732030?*x^3 + 15.1598389616?*x^2 + 6.234592219888?*x + 1.4212119541952?
1.212298506540? 1.661869500173?


In [19]:
AQ1 = mysin(delta) * myinv(mysin(t))
AQ1.print()

-135589.7431848?*x^6 - 9085.96330833?*x^5 + 1166.470182827?*x^4 - 205.3491065340?*x^3 + 25.04630378219?*x^2 - 6.999603936817?*x + 0.9713648040743?
1.476701026516?e6*x^6 - 9085.96330833?*x^5 + 1166.470182827?*x^4 - 205.3491065340?*x^3 + 25.04630378219?*x^2 - 6.999603936817?*x + 0.9713648040743?
0.7562683880462? 1.2439593317765?


In [20]:
beta = (E(11 * pi / 12) - alpha)
beta.print(inDegs=True)

-1*x + 1.439896632895322?
-1*x + 1.439896632895322?
80.5901406828973? 84.409859317103?


In [21]:
W1x = X + mycos(alpha)
W1x.print()

-127978.3951578?*x^6 - 8610.57997279?*x^5 + 1253.007915343?*x^4 - 188.2650535434?*x^3 + 29.29467206147?*x^2 - 6.225445704167?*x + 1.54091255064907?
1.493309533815?e6*x^6 - 8610.57997279?*x^5 + 1253.007915343?*x^4 - 188.2650535434?*x^3 + 29.29467206147?*x^2 - 6.225445704167?*x + 1.54091255064907?
1.3568977281453? 1.7918995266210?


In [22]:
W1y = mysin(alpha)
W1y.print()

0.00108771826850043?*x^5 + 0.0413102025572421?*x^4 - 0.0217543653700086?*x^3 - 0.4957224306869052?*x^2 + 0.130526192220052?*x + 0.9914448613738104?
0.001388888888888889?*x^6 + 0.00108771826850043?*x^5 + 0.0413102025572421?*x^4 - 0.0217543653700086?*x^3 - 0.4957224306869052?*x^2 + 0.130526192220052?*x + 0.9914448613738104?
0.986542328836798? 0.995245788511202?


In [23]:
W2x = mysin(beta)
W2x.print()

-0.00108771826850043?*x^5 + 0.0413102025572421?*x^4 + 0.021754365370009?*x^3 - 0.495722430686906?*x^2 - 0.13052619222006?*x + 0.991444861373811?
0.001388888888888889?*x^6 - 0.00108771826850043?*x^5 + 0.0413102025572421?*x^4 + 0.021754365370009?*x^3 - 0.495722430686906?*x^2 - 0.13052619222006?*x + 0.991444861373811?
0.986542328836798? 0.995245788511202?


In [24]:
W2y = Y + mycos(beta)
W2y.print()

-4.427501585099?e6*x^6 + 13305.6366097?*x^5 + 569.97405679?*x^4 + 286.395491220?*x^3 + 15.0945758654?*x^2 + 7.22603708127?*x + 1.5517381464153?
3.010010145255?e6*x^6 + 13305.6366097?*x^5 + 569.97405679?*x^4 + 286.395491220?*x^3 + 15.0945758654?*x^2 + 7.22603708127?*x + 1.5517381464153?
1.309710135138? 1.825365226914?


In [25]:
Sx = W2x + (W1x - W2x) * E(cos(pi / 3)) + (W2y - W1y) * E(sin(pi / 3))
Sx.print()

-3.898318047468?e6*x^6 + 7217.7278453?*x^5 + 1120.10084977?*x^4 + 153.922961170?*x^3 + 27.9010691924?*x^2 + 2.966906734546?*x + 1.7514069244272?
3.353400019736?e6*x^6 + 7217.7278453?*x^5 + 1120.10084977?*x^4 + 153.922961170?*x^3 + 27.9010691924?*x^2 + 2.966906734546?*x + 1.7514069244272?
1.670783017428? 1.893285720404?


In [26]:
Sy = W2y + (W1x - W2x) * E(sin(pi / 3)) - (W2y - W1y) * E(cos(pi / 3))
Sy.print()

-6.04333920028?e6*x^6 - 804.1612071?*x^5 + 1370.10859364?*x^4 - 19.874290420?*x^3 + 33.0986651364?*x^2 - 1.60007349463?*x + 1.747444481366?
6.51700493050?e6*x^6 - 804.1612071?*x^5 + 1370.10859364?*x^4 - 19.874290420?*x^3 + 33.0986651364?*x^2 - 1.60007349463?*x + 1.747444481366?
1.720134419952? 1.848956879202?


In [27]:
Q2x = mysin(gamma)
Q2x.print()

-636.2680142048?*x^6 + 52.8756859653?*x^5 + 8.920484547049?*x^4 + 0.173259425519?*x^3 - 0.3125871030764?*x^2 + 1.68140232909918?*x + 0.724365086347651?
19817.879824491?*x^6 + 52.8756859653?*x^5 + 8.920484547049?*x^4 + 0.173259425519?*x^3 - 0.3125871030764?*x^2 + 1.68140232909918?*x + 0.724365086347651?
0.66795054433150? 0.78011130258168?


In [28]:
Q2y = Y - mycos(gamma)
Q2y.print()

-4.427003479011?e6*x^6 + 13556.5984560?*x^5 + 616.03768038?*x^4 + 296.193165412?*x^3 + 19.1452865298?*x^2 + 8.00122966013?*x + 0.7317953735413?
3.031582899917?e6*x^6 + 13556.5984560?*x^5 + 616.03768038?*x^4 + 296.193165412?*x^3 + 19.1452865298?*x^2 + 8.00122966013?*x + 0.7317953735413?
0.467999009395? 1.036222661693?


In [29]:
dotprod = Sy - Q2y
dotprod.print()

-9.07492210019?e6*x^6 - 14360.7596631?*x^5 + 754.07091327?*x^4 - 316.067455832?*x^3 + 13.9533786066?*x^2 - 9.60130315475?*x + 1.015649107825?
1.094400840951?e7*x^6 - 14360.7596631?*x^5 + 754.07091327?*x^4 - 316.067455832?*x^3 + 13.9533786066?*x^2 - 9.60130315475?*x + 1.015649107825?
0.685432838951? 1.379436789115?


In [30]:
SQ2 = mysqrt((Sx - Q2x) * (Sx - Q2x) + (Sy - Q2y) * (Sy - Q2y))
SQ2.print()

-1.34686805047?e7*x^6 + 2668.026164?*x^5 + 3112.1522475?*x^4 + 23.76135846?*x^3 + 50.560711606?*x^2 - 5.83713197706?*x + 1.444423084623?
1.53757487611?e7*x^6 + 2668.026164?*x^5 + 3112.1522475?*x^4 + 23.76135846?*x^3 + 50.560711606?*x^2 - 5.83713197706?*x + 1.444423084623?
1.282723022026? 1.721096289958?


In [31]:
diff = myacos(dotprod * myinv2(SQ2)) + E(2 * pi / 3) - E(2) * beta
diff.print(inDegs=True)

-5.8134871253?e7*x^6 - 3330.19254?*x^5 + 131.840565?*x^4 + 183.79150485?*x^3 + 28.494741673?*x^2 + 7.3521788633?*x + 0.005577256879?
3.2942886508?e7*x^6 - 3330.19254?*x^5 + 131.840565?*x^4 + 183.79150485?*x^3 + 28.494741673?*x^2 + 7.3521788633?*x + 0.005577256879?
-16.884350185? 19.171555049?


In [32]:
cond1 = reduceDegree(diff.p, 2)[1]
# we know that diff = 0, so cond1 >= 0 should hold
print(cond1)
cond2 = reduceDegree((AQ1 - E(1)).p, 2)[1]
# we know that AQ1 > 1, so cond2 >= 0 should hold
print(cond2)

75.56118541?*x^2 + 7.3521788633?*x + 0.005577256879?
35.34695682753?*x^2 - 6.999603936817?*x - 0.0286351959258?


In [33]:
# solve p(x) >= 0, -maxx <= x <= maxx,
# where p(x) = ax^2 + bx + c, a > 0
def mysolve(p):
    assert(p.degree(x) == 2)
    a = p.coefficient(x, n=2)
    assert(a > 0)
    b = p.coefficient(x, n=1)
    c = p.coefficient(x, n=0)
    D2 = b ^ 2 - 4 * a * c
    assert(D2 >= 0)
    D = sqrt(D2)
    roots = [(-b - D) / 2 / a, (-b + D) / 2 / a]
    sols = []
    if not(roots[0] <= -maxx):
        sols.append(x <= roots[0])
    if not(roots[1] >= maxx):
        sols.append(x >= roots[1])
    return sols

# solve inequalities cond1 >= 0 and cond2 >= 0
print(mysolve(cond1))
print(mysolve(cond2))

[x >= -0.000764593780?]
[x <= -0.00400978052395?]


In [34]:
# solve the same inequalities using sympy solver
print(solve([cond1 >= 0, -maxx <= x, x <= maxx], x, algorithm='sympy'))
print(solve([cond2 > 0, -maxx <= x, x <= maxx], x, algorithm='sympy'))
print(solve([cond1 >= 0, cond2 > 0, -maxx <= x, x <= maxx], x, algorithm='sympy'))

[[-0.000764593779260685 <= x, x <= 0.0333333333333333]]
[[-0.0333333333333333 <= x, x < -0.00400978052395026]]
False
